# Raytracing of single spherical facet

calin/examples/simulation/vsoptics validation - opst size at 2F.ipynb - Stephen Fegan - 2016-12-06

Copyright 2013, 2016, Stephen Fegan <sfegan@llr.in2p3.fr>
LLR, Ecole polytechnique, CNRS/IN2P3, Universite Paris-Saclay

This file is part of "__calin__". "__calin__" is free software: you can redistribute it and/or modify it under the
terms of the GNU General Public License version 2 or later, as published by
the Free Software Foundation. "__calin__" is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU General Public License for more details.

## Introduction

This notebook tests the spot size from scattering from a single mirror facet by measuring the PSF at 2F.

In [ ]:
%pylab inline
import calin.simulation.vs_optics
import calin.math.vector3d_util

In [ ]:
F = 1600.0
R = F * 2.0
D = 120.0
N = 1000000
D80_2F = 2.8 # Mirror D80 = 28mm at 2F

param = calin.ix.simulation.vs_optics.IsotropicDCArrayParameters()
param.mutable_prescribed_array_layout().add_scope_positions();
dc = param.mutable_reflector()
dc.set_curvature_radius(R)
dc.set_aperture(D)
dc.set_facet_spacing(D)
dc.set_facet_size(dc.facet_spacing())
dc.set_facet_focal_length(R/2.0)
dc.mutable_psf_align().set_object_plane(inf);
dc.set_alignment_image_plane(F)
dc.set_weathering_factor(1.0)
dc.set_facet_spot_size_probability(0.8)
dc.set_facet_spot_size(0.5 * D80_2F) # Set spot size equivalent to D80 at 2F (i.e. half that at 1F)
param.mutable_focal_plane().mutable_translation().set_y(2*F)
param.mutable_pixel().set_spacing(1)
param.mutable_pixel().set_cone_inner_diameter(1)
param.mutable_pixel().set_cone_survival_prob(1)

rng = calin.math.rng.RNG()
cta = calin.simulation.vs_optics.VSOArray()
cta.generateFromArrayParameters(param, rng)
scope = cta.telescope(0)
    
raytracer = calin.simulation.vs_optics.VSORayTracer(cta, rng)
ph = calin.math.ray.Ray()
info = calin.simulation.vs_optics.VSOTraceInfo()

beam_dir = scope.opticalAxis()
beam_dir *= -1.0
beam_cen = asarray([0,0,0]) - beam_dir*R

x = []
y = []
for i in range(0,N):
    raytracer.fanBeam(ph, beam_cen, beam_dir, D/cos(30.0/180.*pi)/R*0.52, 300)
    pixel = raytracer.trace(ph, info, scope)
    if info.rayHitFocalPlane():
        x.append(info.fplane_z)
        y.append(info.fplane_x)
x = array(x)
y = array(y)


In [ ]:
print("Raytraced D80 at 2F: %.3f cm"%(percentile(sqrt(x**2+y**2),80) * 2.0)) # Diameter D80